# Prerequisites

In [ ]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project/validation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ece1786-project/validation


In [ ]:
import transformers
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd
import torch 
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import evaluate
import numpy as np

In [ ]:
# model_checkpoint = "gpt2" #"distilgpt2"
# tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)
# filename = "drink_val.pkl"
drinks_file = "drink_val.pkl"
cakes_file = "cake_val.pkl"
drink_df = pd.read_pickle(drinks_file)
cake_df = pd.read_pickle(cakes_file)
device = torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

#Functions

In [ ]:
def create_propmts(df, index):
    recipe = "RECIPE NAME\n" + df.loc[index, "Name"] + " \n\nRECIPE INGREDIENTS\n"
    prompts = [recipe]
    for ingredient in df.loc[index, "Ingredients"]:
      recipe += ingredient + "\n"
    recipe += "\nRECIPE INSTRUCTIONS\n" 
    for instruction in  df.loc[index, "Instructions"]:
      recipe += instruction + "\n"
    answers = [[recipe]]
    return answers, prompts

In [ ]:
def score(metric, df, index, model):
  answers, prompts = create_propmts(df, index)
  attempts = []
  encoded_prompt = tokenizer(prompts, return_tensors="pt").input_ids.to(device)
  ground_truth_length = (tokenizer(answers[0], return_tensors="pt").input_ids.to(device)).size(1)
  print(f"tokenized answer length: {ground_truth_length}")
  generated_sequence = model.generate(encoded_prompt, do_sample=True, max_length=ground_truth_length, temperature = 0.5, return_dict_in_generate=True, output_scores=True)
  decoded_sequence = tokenizer.batch_decode(generated_sequence.sequences, skip_special_tokens=True)[0]
  attempts.append(decoded_sequence)
  
  eval = evaluate.load(metric)
  results = eval.compute(predictions=attempts, references=answers)
  return results, decoded_sequence

In [ ]:
def run(df, model, iterations=5, cardinality=10):
  predictions = []
  scores = np.zeros((cardinality, iterations))
  for i in range(cardinality):
    tmp = []
    for j in range(iterations):
      s, r = score("bleu", df, i, model)
      scores[i][j] = s["bleu"]
      tmp.append(r)
    predictions.append(tmp[np.argmax(scores[i])])
  return predictions, scores

#Drinks

In [ ]:
drinkmodel = AutoModelForCausalLM.from_pretrained("../gpt2model_drink")

In [ ]:
drink_answers, drink_scores = run(drink_df, drinkmodel)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 262


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 262


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

tokenized answer length: 124


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 124


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 124


In [ ]:
with open('drink_val_results.txt', 'w') as fp:
    for item in drink_answers:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
np.mean(drink_scores)

0.18769332673442465

In [ ]:
print(drink_answers[-1])

RECIPE NAME
Cider Delighter 

RECIPE INGREDIENTS
1 1/2 ounces gin
1/2 ounce lemon juice, freshly squeezed
1/2 ounce simple syrup
Garnish: lemon twist

RECIPE INSTRUCTIONS
Add the gin, lemon juice and simple syrup into a shaker with ice and shake until well-chilled.
Strain into a chilled coupe glass.
Garnish with a lemon twist.



# Cake

In [ ]:
cakemodel = AutoModelForCausalLM.from_pretrained("../gpt2-finetuned-cake/checkpoint-9000")

In [ ]:
cake_answers, cake_scores = run(cake_df, cakemodel)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 305


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 305


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

tokenized answer length: 448


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 448


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tokenized answer length: 448


In [ ]:
with open('cake_val_results.txt', 'w') as fp:
    for item in cake_answers:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [ ]:
np.mean(cake_scores)

0.14748013450765454

In [ ]:
print(cake_answers[-1])

RECIPE NAME
Ricotta Pie 

RECIPE INGREDIENTS
2 cups all-purpose flour
1 ½ teaspoons baking powder
1 ½ teaspoons baking soda
½ cup butter, softened
1 cup chopped pecans
2  eggs
1 cup flaked coconut
1 teaspoon salt
1 teaspoon vanilla extract
1 ¼ cups vegetable oil
2 cups white sugar

RECIPE INSTRUCTIONS
Preheat oven to 350 degrees F. Grease and flour two 9-inch round cake pans.
Combine flour, baking soda, baking powder, and salt in a large bowl. Beat in eggs, 1 at a time, blending well after each. Stir in coconut and vanilla.
Pour flour mixture into prepared pans. Bake in the preheated oven until a toothpick inserted into the center comes out clean, about 20 minutes. Cool on wire racks for 10 minutes.
While cakes cool, make icing: Combine pecans, sugar, oil, eggs, vanilla, and flour in a large bowl. Beat until fluffy.
Spread icing over cooled cake layers. Refrigerate until set, about 1 hour.

